## UNHR

In [ ]:
%load_ext autoreload
%autoreload 2

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import tqdm
import os
import pandas as pd


from src.preprocessing.hatespeech_dataset_querying import hatespeech_v2_load_train_and_validation_set, map_predicted_to_label_hatesv2
from src.preprocessing.toxigen_querying import load_toxigen_train_and_validation

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../..")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"## UNHR

In [ ]:
train_df, validation_df = hatespeech_v2_load_train_and_validation_set()
train_df

In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained("unhcr/hatespeech-detection")
model = AutoModelForSequenceClassification.from_pretrained("unhcr/hatespeech-detection")
# Use a pipeline as a high-level helper
pipe = pipeline("text-classification", model="unhcr/hatespeech-detection", device="cuda:0")  # 

y_pred = pipe(list(train_df["text"].values))

In [ ]:
from sklearn.metrics import classification_report

y_truth = train_df["label"]
mapped_pred = map_predicted_to_label_hatesv2(y_pred)

print(classification_report(y_truth, mapped_pred))

In [ ]:
# y_pred

## Retraining the whole model for our dataset

In [ ]:
# prepare the dataset in the correct format
import datasets

def convert_train_and_validation_to_dataset(train_df, validation_df):
    train_dataset = datasets.Dataset.from_pandas(df=train_df.iloc[:100], split="train")
    validation_dataset = datasets.Dataset.from_pandas(df=validation_df.iloc[:100], split="validation")
    ds = datasets.DatasetDict()
    ds["train"] = train_dataset
    ds["validation"] = validation_dataset
    return ds

hate_dataset_dict = convert_train_and_validation_to_dataset(train_df, validation_df)
hate_dataset_dict

In [ ]:
# imports needed for retraining 
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import torch

torch.cuda.empty_cache()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unhcr/hatespeech-detection")
model = AutoModelForSequenceClassification.from_pretrained("unhcr/hatespeech-detection")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

tokenized_datasets = hate_dataset_dict.map(tokenize_function, batched=True)
training_args = TrainingArguments(output_dir="test_trainer")
metric = evaluate.load("accuracy")

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(10))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()